## Project 4 - Notebook 2 - EDA (by Wilson Alberto Torres)

In [159]:
# Importing libraries
import pandas as pd
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [160]:
# Importing data set created in previous web scrapping stage
# In this notebook, exploratory data analysis (EDA) is applied
# and will produce two data sets ready to be used for exploring 
# machine learning modelling.
result_df = pd.read_csv('jobs_project4v2.csv')
result_df2 = pd.read_csv('jobs_project4v2.csv')

In [161]:
result_df.head(2)

,s_keyword,s_loc,range_sal,job_title,job_id,city,job_agency_url,job_agency,job_type,job_salary,job_field,job_subfield,job_des,job_url,job_date,job_full_desc
0,data-analyst,All-Melbourne-VIC,30000-60000,Excel Data Analyst,50354487,Melbourne,https://www.seek.com.au/Natural-Selection-Grou...,Natural Selection Group,This is a Contract/Temp job,no info,Information & Communication Technology,Other,Working with excel on customer quoting initiat...,https://www.seek.com.au/job/50354487?type=stan...,6 Aug 2020,Our Client has a new opportunity for a Excel D...
1,data-analyst,All-Melbourne-VIC,30000-60000,Product Analytics Specialist,50392356,Melbourne,https://www.seek.com.au/Openpay-jobs,Openpay Pty Ltd,This is a Full Time job,no info,Information & Communication Technology,Product Management & Development,Openpay (ASX: OPY) is an omni-channel payment ...,https://www.seek.com.au/job/50392356?type=stan...,13 Aug 2020,Product Analytics Specialist\nOpenpay (ASX: OP...


In [162]:
result_df.nunique()

s_keyword            3
s_loc                2
range_sal            3
job_title         4455
job_id            5898
city                 2
job_agency_url    2228
job_agency        2295
job_type             4
job_salary        1409
job_field           29
job_subfield       258
job_des           5288
job_url           5898
job_date            70
job_full_desc     5529
dtype: int64

In [163]:
pd.set_option('display.max_colwidth',200)
pd.set_option('display.max_rows',2000)

In [164]:
result_df[result_df['job_salary'] != 'no info'][['job_salary']];

In [165]:
# Filter by Job_id
result_df[result_df.job_id == 50276217]

,s_keyword,s_loc,range_sal,job_title,job_id,city,job_agency_url,job_agency,job_type,job_salary,job_field,job_subfield,job_des,job_url,job_date,job_full_desc
2468,data-analyst,All-Sydney-NSW,70000-100000,Financial Analyst,50276217,Sydney,https://www.seek.com.au/Robert-Walters-jobs,Robert Walters,This is a Full Time job,"$97,000k - $107,500k p.a. + + super",Accounting,Analysis & Reporting,"Opportunity to advance your career, this role will provide an interface between the business operations and the central Finance team.",https://www.seek.com.au/job/50276217?type=standard#searchRequestToken=8ca2a69c-99c9-4da1-b3b2-41beb21f9c36,22 Jul 2020,"Financial Analyst\nThis role will provide an interface between the business operations and the central Finance team and other interested parties, including Treasury, and will provide relevant fina..."


In [166]:
# Filter by Index
result_df[result_df.index == 2227]

,s_keyword,s_loc,range_sal,job_title,job_id,city,job_agency_url,job_agency,job_type,job_salary,job_field,job_subfield,job_des,job_url,job_date,job_full_desc
2227,data-analyst,All-Sydney-NSW,70000-100000,Reporting Analyst / VBA Specialist: $125k Package + Parking,50350376,Sydney,https://www.seek.com.au/Reo-Group-jobs,Reo Group,This is a Full Time job,$125 k pack + 5 weeks annual leave,Accounting,Analysis & Reporting,Join a Multi-billion dollar household brand to lead the development and automation of dashboard reporting. Additional benefits + view to permanent.,https://www.seek.com.au/job/50350376?type=standout#searchRequestToken=298c5961-9be3-4cc8-b1ff-c38ed7e4abe7,5 Aug 2020,Reporting Analyst / VBA Specialist: $125k Package + Parking\nLocation: Western Sydney\nSalary: $120 - 125k Package + Benefits + Parking\nRole Benefits: be part of a large and growing organisation ...


In [167]:
#pd.set_option('display.max_colwidth',50)
result_df.shape

(5898, 16)

## Cleaning Salary Column

In [168]:
temp = result_df[result_df['job_salary'] != 'no info'][['job_salary']]
print(temp.shape)

(1945, 1)


In [169]:
temp;

In [170]:
tmp = temp['job_salary'].replace(to_replace=r'[\$a-jl-zA-JL-Z\'\:\|\`\,\@\/\(\)\!\+\&\s]', value='', regex=True);
#tmp = temp['job_salary'].replace(to_replace=r'(\d+,\d+)|(\d+[kK])', value='', regex=True)
#re.findall(r"(\d+,\d+)|(\d+[kK])",temp['job_salary'])
tmp;

#'\d to \d'
#(\d+)

In [171]:
#temp['job_salary'].str.extractall('(\d+)').unstack();

In [172]:
#temp['job_salary'].replace(to_replace='\,', value='', regex=True).str.extractall('(\d+)k').unstack();

In [173]:
temp['job_salary'] = temp['job_salary'].str.replace(' k',',000')
temp['job_salary'] = temp['job_salary'].str.replace(' K',',000')
temp['job_salary'] = temp['job_salary'].str.replace('k',',000')
temp['job_salary'] = temp['job_salary'].str.replace('K',',000')

In [ ]:
temp1 = temp['job_salary'].replace(to_replace='\,', value='', regex=True).str.extractall('(\d{5,})').unstack()
temp1

In [175]:
temp1.shape

(1026, 4)

In [176]:
result_df2.loc[temp1.index,'cleanedsalary'] = temp1[0][0]
result_df2.head(5)

,s_keyword,s_loc,range_sal,job_title,job_id,city,job_agency_url,job_agency,job_type,job_salary,job_field,job_subfield,job_des,job_url,job_date,job_full_desc,cleanedsalary
0,data-analyst,All-Melbourne-VIC,30000-60000,Excel Data Analyst,50354487,Melbourne,https://www.seek.com.au/Natural-Selection-Group-jobs,Natural Selection Group,This is a Contract/Temp job,no info,Information & Communication Technology,Other,Working with excel on customer quoting initiatives,https://www.seek.com.au/job/50354487?type=standout#searchRequestToken=3d01f78b-c5b9-4fcf-a895-d098a1fcf68f,6 Aug 2020,Our Client has a new opportunity for a Excel Data Analyst for a period of 5 weeks. The role will be working remotely with a laptop supplied. This role will be responsible for capturing & accuratel...,NaN
1,data-analyst,All-Melbourne-VIC,30000-60000,Product Analytics Specialist,50392356,Melbourne,https://www.seek.com.au/Openpay-jobs,Openpay Pty Ltd,This is a Full Time job,no info,Information & Communication Technology,Product Management & Development,"Openpay (ASX: OPY) is an omni-channel payment solution that provides customers with flexible, interest-free payment plans.",https://www.seek.com.au/job/50392356?type=standard#searchRequestToken=3d01f78b-c5b9-4fcf-a895-d098a1fcf68f,13 Aug 2020,"Product Analytics Specialist\nOpenpay (ASX: OPY) is an omni-channel payment solution that provides customers with flexible, interest-free payment plans, in-store, online and through the Openpay Ap...",NaN
2,data-analyst,All-Melbourne-VIC,30000-60000,"Graduate Database Developer/Analyst (permanent, Melb Metro)",50374020,Melbourne,https://www.seek.com.au/jobs?advertiserid=30926157,MIEPOL,This is a Full Time job,no info,Information & Communication Technology,Developers/Programmers,We are seeking a Graduate Database Developer/Analyst with the ability to develop SQL code/queries and use reporting tools & understanding of Powerbi,https://www.seek.com.au/job/50374020?type=standard#searchRequestToken=3d01f78b-c5b9-4fcf-a895-d098a1fcf68f,11 Aug 2020,"We are looking for a Graduate Database Developer/Analyst for our West Footscray head office. You will have the ability with developing SQL code/queries and reporting tools, understanding of Powerb...",NaN
3,data-analyst,All-Melbourne-VIC,30000-60000,Graduate Analyst Position - Melbourne,50372185,Melbourne,https://www.seek.com.au/Proforce-jobs,Proforce,This is a Full Time job,"$50,000 per annum",Sales,Analysis & Reporting,Graduate inventory and sales analyst position with a focus on forecasting and analysis. Great opportunity to join this Melbourne based business.,https://www.seek.com.au/job/50372185?type=standout#searchRequestToken=3d01f78b-c5b9-4fcf-a895-d098a1fcf68f,10 Aug 2020,"The Business\n\nFounded in 2014 this Australian owned company has become Australia’s fastest growing home appliance wholesaler/distributor, servicing a number of Australia’s largest retailers and ...",50000
4,data-analyst,All-Melbourne-VIC,30000-60000,Business Analyst Junior or Emerging,50384667,Melbourne,https://www.seek.com.au/Interface-Recruitment-jobs,Interface Recruitment,This is a Full Time job,$60 - $65 plus super great ground floor role,Information & Communication Technology,Business/Systems Analysts,Want to join a medium sized Software House with friendly people and a great culture? Be a part of exceptional growth around a niche Product into Govt.,https://www.seek.com.au/job/50384667?type=standard#searchRequestToken=3d01f78b-c5b9-4fcf-a895-d098a1fcf68f,12 Aug 2020,"Business Analyst Junior or Emerging\nInterface Recruitment is a full service, Melbourne based specialist recruitment agency and Microsoft Partner. We specialise in contract and permanent placement...",NaN


In [177]:
#Cleaning inconsistent salary value
result_df2['cleanedsalary'] = result_df2['cleanedsalary'].str.replace('150000000','150000')
result_df2['cleanedsalary'] = result_df2['cleanedsalary'].str.replace('97000000','97000')

In [178]:
result_df3 = result_df2[['job_id','s_keyword','job_title','city','job_type','job_field','job_subfield','range_sal','cleanedsalary','job_full_desc']]

result_df4 = result_df2[['job_id','s_keyword','job_title','city','job_type','job_field','job_subfield','range_sal','cleanedsalary','job_full_desc']]

In [179]:
result_df3.head(1);

In [181]:
result_df3.shape, result_df4.shape

((5898, 10), (5898, 10))

In [182]:
result_df3['cleanedsalary'].isnull().value_counts()

True     4872
False    1026
Name: cleanedsalary, dtype: int64

In [183]:
result_df3.isnull().sum()

job_id              0
s_keyword           0
job_title           0
city                0
job_type            0
job_field           0
job_subfield        0
range_sal           0
cleanedsalary    4872
job_full_desc       0
dtype: int64

In [184]:
result_df3['s_keyword'].unique()

array(['data-analyst', 'data-science', 'data-engineer'], dtype=object)

In [185]:
result_df3.groupby(['s_keyword'])[['job_id']].count()

,job_id
s_keyword,
data-analyst,4922
data-engineer,588
data-science,388


In [186]:
# Dropping rows with NaNs
result_df3.dropna(inplace=True)
result_df3.head(1);

<ipython-input-186-454104fcd661>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3.dropna(inplace=True)


In [187]:
result_df3.shape

(1026, 10)

In [188]:
result_df3[result_df3['cleanedsalary'] == '0414571736']

,job_id,s_keyword,job_title,city,job_type,job_field,job_subfield,range_sal,cleanedsalary,job_full_desc
4287,50343084,data-analyst,"Company Secretary, Data Privacy and Compliance Manager",Sydney,This is a Full Time job,Legal,Generalists - In-house,120000-200000,0414571736,"We have been briefed exclusively to partner with Danone, One Planet. One Health to source a suitably qualified Company Secretary, Data Privacy Officer and Compliance Manager due to the imminent mo..."


In [189]:
result_df3.shape

(1026, 10)

In [190]:
# Dropping row with inconsistent salary
result_df3.drop(index=4287, inplace=True)

/opt/anaconda3/envs/gres/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [191]:
result_df3.shape

(1025, 10)

## Cleaning job_full_desc column

In [150]:
result_df3.reset_index(inplace=True)

In [151]:
result_df3.head(1)

,index,job_id,s_keyword,job_title,city,job_type,job_field,job_subfield,range_sal,cleanedsalary,job_full_desc
0,3,50372185,data-analyst,Graduate Analyst Position - Melbourne,Melbourne,This is a Full Time job,Sales,Analysis & Reporting,30000-60000,50000,"The Business\n\nFounded in 2014 this Australian owned company has become Australia’s fastest growing home appliance wholesaler/distributor, servicing a number of Australia’s largest retailers and ..."


In [152]:
pd.set_option('display.max_colwidth',50)

In [ ]:
# Cleaning DataSet for question 1
result_df3['job_full_desc'].replace(to_replace='\n\n|\n|\/', value=' ', regex=True, inplace=True)
result_df3['job_full_desc'].replace(to_replace='[(\d+)|\[\]\$\(\)\!\%]', value='', regex=True, inplace=True)
result_df3['job_full_desc'].str.lower()

In [ ]:
# Cleaning DataSet for question 2
result_df4['job_full_desc'].replace(to_replace='\n\n|\n|\/', value=' ', regex=True, inplace=True)
result_df4['job_full_desc'].replace(to_replace='[(\d+)|\[\]\$\(\)\!\%]', value='', regex=True, inplace=True)
result_df4['job_full_desc'].str.lower()

In [196]:
result_df3[result_df3.cleanedsalary == 97000000]

,job_id,s_keyword,job_title,city,job_type,job_field,job_subfield,range_sal,cleanedsalary,job_full_desc


In [197]:
result_df3['cleanedsalary'] = result_df3['cleanedsalary'].apply(lambda x: int(x))

<ipython-input-197-163a9a1afbce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df3['cleanedsalary'] = result_df3['cleanedsalary'].apply(lambda x: int(x))


In [198]:
result_df3.dtypes

job_id            int64
s_keyword        object
job_title        object
city             object
job_type         object
job_field        object
job_subfield     object
range_sal        object
cleanedsalary     int64
job_full_desc    object
dtype: object

In [199]:
result_df3['cleanedsalary'].unique()

array([ 50000,  60000,  55000,  65325,  65000,  54000,  51000,  45000,
        85000,  40000, 100000,  85279,  98396,  80000,  90000,  70000,
        67195, 120000, 115000,  88004,  98744, 125000,  80669,  75000,
        69401, 105000,  93000,  95000,  86434,  86000,  96527, 101955,
        89314,  91250, 110000,  84930,  94471, 135000, 140000, 180000,
       150000, 116156, 160000, 130000, 188381, 165000, 200000, 137000,
       170000, 300000, 147804, 250000,  17000,  56000,  67000,  64000,
        58000,  29000,  79000,  63000,  63670,  35000,  12000, 107865,
        97000,  97152,  90743, 190000,  87000,  77000,  70146, 108147,
       109000,  93266, 109224,  98000,  79452,  82208,  82178,  73160,
        74233, 144126, 127559, 220000, 120200, 118015, 110745, 128089,
       113735, 240000, 108000, 125444, 145000, 127384, 124752, 155000,
        10000,  84279,  88994,  97812, 103409,  68000,  89766,  73000])

In [205]:
result_df4.drop(columns='cleanedsalary', axis=1, inplace=True)
result_df4.dtypes

/opt/anaconda3/envs/gres/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


job_id            int64
s_keyword        object
job_title        object
city             object
job_type         object
job_field        object
job_subfield     object
range_sal        object
job_full_desc    object
dtype: object

In [206]:
# Creating a CSV file, ready for modeling in question 1
result_df3.to_csv('jobs_project4-modelQ1.csv',index=False)
print(result_df3.shape)

# Creating a CSV file, ready for modeing in question 2
result_df4.to_csv('jobs_project4-modelQ2.csv',index=False)
print(result_df4.shape)

(1025, 10)
(5898, 9)
